# 2.1 Neural Network 

In [47]:
import numpy as np

In [48]:
# first two bit A and B, remaining 3 bit is operator identifier
X = np.array([
            #AND 
            [0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0],
            [1, 0, 0, 0, 0],
            [1, 1, 0, 0, 0],
            #OR    
            [0, 0, 0, 0, 1],
            [0, 1, 0, 0, 1],
            [1, 0, 0, 0, 1],
            [1, 1, 0, 0, 1],
            #XOR
            [0, 0, 0, 1, 0],
            [0, 1, 0, 1, 0],
            [1, 0, 0, 1, 0],
            [1, 1, 0, 1, 0],
            #NAND
            [0, 0, 0, 1, 1],
            [0, 1, 0, 1, 1],
            [1, 0, 0, 1, 1],
            [1, 1, 0, 1, 1],
            #NOR
            [0, 0, 1, 0, 0],
            [0, 1, 1, 0, 0],
            [1, 0, 1, 0, 0],
            [1, 1, 1, 0, 0],
            ]).T

X.shape

(5, 20)

In [49]:
#target output        AND         OR           XOR       NAND         NOR
Y = np.array([[0, 0, 0, 1],[0, 1, 1, 1], [0, 1, 1, 0],[1, 1, 1, 0], [1, 0, 0, 0]])
Y = Y.reshape(1,20)
Y.shape

(1, 20)

## Sigmoid as Activation Function
 *  Ensures that the network is capable of computing all sorts of complex functions
 *  it helps push the output towards 0 or 1 (classifying into 2 classes)

In [50]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [51]:
# Initialize weights and bias 
def init_weights(input_features, n_hidden, output_features):
    # Weights 
    W1 = np.random.randn(n_hidden, input_features)
    W2 = np.random.randn(output_features, n_hidden)
    
    # bias 
    b1 = np.zeros((n_hidden, 1))
    b2 = np.zeros((output_features, 1))

    
    weights = {"W1" : W1, "b1" : b1, 
               "W2" : W2, "b2" : b2}
    return weights 

#### Forward Propagation
* forward pass , calculation and storage of intermediate variables for neural network

In [52]:
#used to perform forward propagation
def forward_propagation(X, Y, weights):
    m = X.shape[1] #number of samples
    
    #extract weights 
    W1 = weights["W1"]
    b1 = weights["b1"]
    W2 = weights["W2"]
    b2 = weights["b2"]
    
    #calculate output
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    #save the parameters 
    params = (Z1, A1, W1, b1, Z2, A2, W2, b2)
    
    #calculate cost function 
    cost = np.log(A2)* Y + np.log(1 - A2)*(1 - Y)
    cost = -np.sum(cost) / m

    return cost, params, A2

#### Backward Prpagation
* Calulating gradient of neural network parameters 

In [53]:
# used to perform Backward propagation
def backward_propagation(X, Y, params):
    m = X.shape[1]
    #extract parameters 
    (Z1, A1, W1, b1, Z2, A2, W2, b2) = params
    
    #calculate output error
    dZ2 = A2 - Y
    
    #caculate gradient of second weights 
    dW2 = np.dot(dZ2, A1.T) / m  
    db2 = np.sum(dZ2, axis = 1, keepdims = True )
    
    #calculates error of hidden layer 
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * A1 * (1 - A1)
    
    #find gradients of first weights
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis = 1, keepdims = True) / m
    
    #store gradients 
    gradients = {"dW2": dW2, "db2" : db2,
                 "dW1": dW1, "db1": db1 }
    
    return gradients 


#### Updating Weights using gradients as well as learning rate 

In [54]:
#used to update weights, lr = learning rate 
def update_parameters(weights, gradients, lr): 
    weights["W1"] = weights["W1"] - lr * gradients["dW1"]
    weights["b1"] = weights["b1"] - lr * gradients["db1"]
    weights["W2"] = weights["W2"] - lr * gradients["dW2"]
    weights["b2"] = weights["b2"] - lr * gradients["db2"]
    
    return weights 

In [55]:
#Configure parameters 
n_hidden = 5 # number of neurons in hidden layer
input_features = X.shape[0] #number of input neurons
output_features = Y.shape[0] 
weights = init_weights(input_features, n_hidden, output_features) 
epochs = 10000
lr = 0.01
cost = np.zeros((epochs, 1))

In [56]:
#train network 
for i in range(epochs):
    cost[i, 0], params, A = forward_propagation(X, Y, weights)
    gradients = backward_propagation(X, Y, params)
    weights = update_parameters(weights, gradients, lr)

In [57]:
#testing on training set 
cost, _, A = forward_propagation(X, Y, weights ) #predict values 
y_pred = (A > 0.5) * 1.0 #find labels from probabilities 
#calculate accuracy 
score = 1 - np.mean(np.abs(y_pred - Y))
print("Accuracy :{} %".format(score*100)) 

Accuracy :90.0 %


In [58]:
# sample testing 
X_test = np.array([[1,1,0,0,0], #AND 
                   [1,1,0,0,1], #OR
                   [1,1,0,1,0], #XOR
                   [1,1,0,1,1], #NAND
                   [1,1,1,0,0]  #NOR
                   ])
y_true = np.array([1, 1, 0, 0,0])  # target values for sample input
_,_, A2 = forward_propagation(X_test, y_true, weights)
y_pred = (A2 > 0.5) * 1
print(y_pred)

[[0 0 0 1 1]]
